## Preprocessing 01 (Working with Sampling)
#### 1. Crawling
#### 2. Tranformation included Cleansing & Denoising
#### 3. Imputation

In [1]:
import sys
import ssl
from urllib.request import Request, urlopen
from datetime import datetime

def crawling(url='', encoding='utf-8', err=lambda e: print(f'{e} : {datetime.now()}', file=sys.stderr)):
    try:
        req = Request(url)
        ssl._create_default_https_context = ssl._create_unverified_context

        resp = urlopen(req)
        recv = resp.read().decode(encoding, errors='replace')

        print(f'{datetime.now()}: success for request [{url}]')
        return recv
    except Exception as e:
        err(e)

In [2]:
from functools import reduce
from itertools import count

resfetch = []
for idx in count(start=1):
    l = len(resfetch)
    resfetch[l:] = list(filter(
        lambda s: s.startswith('#'),
        reduce(
            lambda param, func: func(param),
            [lambda txt: [] if txt is None else txt.splitlines(), lambda strs: [f(s) for f in [str.strip] for s in strs]],
            crawling(
                'https://raw.githubusercontent.com/kickscar/HELLCHANG-PRACTICES/main/training-record/Week0%02d.md' % idx,
                err=lambda e: None
            )
        )
    ))
    if len(resfetch) == l:
        break

print(resfetch)

2022-08-27 16:02:18.119935: success for request [https://raw.githubusercontent.com/kickscar/HELLCHANG-PRACTICES/main/training-record/Week001.md]
2022-08-27 16:02:18.162808: success for request [https://raw.githubusercontent.com/kickscar/HELLCHANG-PRACTICES/main/training-record/Week002.md]
2022-08-27 16:02:18.201191: success for request [https://raw.githubusercontent.com/kickscar/HELLCHANG-PRACTICES/main/training-record/Week003.md]
2022-08-27 16:02:18.239687: success for request [https://raw.githubusercontent.com/kickscar/HELLCHANG-PRACTICES/main/training-record/Week004.md]
2022-08-27 16:02:18.279503: success for request [https://raw.githubusercontent.com/kickscar/HELLCHANG-PRACTICES/main/training-record/Week005.md]
2022-08-27 16:02:18.328530: success for request [https://raw.githubusercontent.com/kickscar/HELLCHANG-PRACTICES/main/training-record/Week006.md]
2022-08-27 16:02:18.376938: success for request [https://raw.githubusercontent.com/kickscar/HELLCHANG-PRACTICES/main/training-reco

In [3]:
import re

restransform = []
for res in resfetch:
    if res.startswith('####'):
        name, record = (s.strip() for s in res.replace('####', '').strip().split(':'))
        # sval, unit = re.compile(r'(\d+.\d*)\s*(.*)').match(re.sub(r"[\([{})\]\s]", "", val)).groups()
        restransform[len(restransform):] = [(date, re.sub(r'[\d+\\.]', '', name.strip()).strip().title(), *re.split(r'\s+', record.strip()))]
    elif res.startswith('##'):
        date = res.replace('##', '').strip()

print(restransform)

[('2021/06/24', 'Flat Bench Press[Machine, Smith]', '40kg', '12reps', '5sets'), ('2021/06/24', 'Seated Chest Press[Machine]', '20kg', '12reps', '5sets'), ('2021/06/24', 'Lat Pulldown[Machine]', '20kg', '12reps', '5sets'), ('2021/06/24', 'Seated Low Row[Cable]', '15kg', '12reps', '3sets'), ('2021/06/24', 'Assisted Pullup[Machine]', '-50kg', '12reps', '3sets'), ('2021/06/24', 'Shoulder Press[Machine]', '5kg', '12reps', '5sets'), ('2021/06/25', 'Low Pulley[Machine]', '20kg', '12reps', '5sets'), ('2021/06/25', 'Assisted Pullup[Machine]', '-50kg', '12reps', '5sets'), ('2021/06/25', 'Shoulder Press[Machine]', '15kg', '12reps', '5sets(5th', 'failed)'), ('2021/06/25', 'Leg Extension[Machine]', '20kg', '12reps', '3sets,', '25kg', '12reps', '2sets,', '30kg', '10reps', '2sets'), ('2021/06/25', 'Lying Leg Curls[Machine]', '20kg', '12reps', '5sets(5th', 'failed)'), ('2021/06/25', 'Side Lateral Raise[Dumbbell]', '4kg', '12reps', '5sets'), ('2021/06/28', 'Flat Bench Press[Machine, Smith]', '40kg', '1

In [4]:
# Which Exercise?

exs = [t[1] for t in restransform]
dataset_exs = [(ex, exs.count(ex)) for ex in list(set(exs))]

sorted_by_ex = sorted(dataset_exs, key=lambda t: t[0])
for d in sorted_by_ex:
    print(d)

('Arnold Press[Dumbbell]', 16)
('Assisted Dip[Machine]', 14)
('Assisted Dips[Machine]', 24)
('Assisted Pullup[Machine]', 55)
('Back Extension', 36)
('Basic Squat', 4)
('Basic Squats', 1)
('Behind The Neck Press[Machine, Smith]', 3)
('Bent Over Lateral Raise[Dumbbell]', 2)
('Bent Over Row[Barbell]', 5)
('Bent Over Row[Dumbbell ]', 1)
('Bent Over Row[Dumbbell]', 13)
('Biceps Curls [Ezbar]', 1)
('Biceps Curls [Fixed Barbell]', 1)
('Biceps Curls [Machine]', 13)
('Cable Chest Fly[Cable]', 1)
('Cable Cross-Over[Cable]', 4)
('Cable Triceps Pushdown[Cable]', 8)
('Chest Fly [Machine, Pec Deck]', 2)
('Chest Fly[Machine, Pec Deck]', 37)
('Chest Press[Machine]', 2)
('Cross Lunges', 1)
('Dead Bug', 2)
('Deadlift[Barbell]', 83)
('Deadlift[Dumbbell]', 8)
('Deadlift[Kettlebell]', 12)
('Decline Bench Press[Barbell]', 10)
('Dips', 18)
('Face Pulls[Cable]', 1)
('Flat Bench Press[Barbell]', 52)
('Flat Bench Press[Dumbbel]', 1)
('Flat Bench Press[Dumbbell]', 2)
('Flat Bench Press[Free]', 1)
('Flat Bench Pr

In [5]:
# and How Often?
sorted_by_count = sorted(dataset_exs, key=lambda t: t[1], reverse=True)
for d in sorted_by_count:
    print(d)

('Lying Leg Curls[Machine]', 111)
('Lat Pulldown[Machine]', 98)
('Plank', 92)
('Seated Chest Press[Machine]', 86)
('Deadlift[Barbell]', 83)
('Hip Adduction[Machine]', 81)
('Hip Abduction[Machine]', 73)
('Shoulder Press[Machine]', 73)
('Leg Extension[Machine]', 62)
('Seated Row[Machine]', 62)
('Side Lateral Raise[Dumbbell]', 60)
('Assisted Pullup[Machine]', 55)
('Flat Bench Press[Barbell]', 52)
('Power Leg Press[Machine]', 50)
('Straight Arm Pulldown[Cable]', 39)
('Chest Fly[Machine, Pec Deck]', 37)
('One Leg Deadlift[Kettlebell]', 36)
('Back Extension', 36)
('Running', 33)
('Seated Low Row[Cable]', 33)
('Wide Pulldown Rear[Machine]', 33)
('Side-Lying Hip Abduction', 32)
('Incline Bench Press[Machine, Smith]', 25)
('Assisted Dips[Machine]', 24)
('Wrist Roller', 22)
('Incline Chest Press[Machine]', 19)
('Long Pull Row[Cable]', 19)
('Dips', 18)
('Incline Bench Press[Barbell]', 17)
('Incline Bench Press[Machine]', 16)
('Reverse Pec Deck Fly[Machine, Pec Deck]', 16)
('Arnold Press[Dumbbell]